# EXO

### importer les données json dans votre base de données

In [138]:
import pandas as pd
from pymongo import MongoClient
import json
import streamlit as st
from bson.objectid import ObjectId
import re

In [20]:
client = MongoClient("localhost", 4344)
db = client.library_repository
table = db.libraries

In [15]:
# Lecture du json
df = pd.read_json("./repertoire-bibliotheques.json")


In [106]:
# Import des données
if table.count_documents({}) == 0:    
    records = df.to_dict(orient="records")
    table.insert_many(records)
else:
    print("Table déja alimenté")

In [32]:
list(table.find_one())

['_id', 'datasetid', 'recordid', 'fields', 'geometry', 'record_timestamp']

### combien de bibliotheques se trouve a paris

In [40]:
librariesInParis = table.aggregate([
    { "$match": {
        "fields.commune": "Paris"
    }},
    { "$group": {
        "_id": "Compteur", "Nombre de librairies à Paris": { "$sum": 1}
    }}
])

list(librariesInParis)

[{'_id': 'Compteur', 'Nombre de librairies à Paris': 615}]

### Faites un update sur le champs "commune" pour le passer en lower case et refaite le compte des bibliotheques a paris

In [109]:
for doc in table.find(no_cursor_timeout=True):
    pk = ObjectId(str(doc.get("_id")))
    fields = doc.get('fields')
    city = fields.get('commune')
    if city:
        city = city.lower()
        table.update_one({"_id": pk}, {"$set":{"fields.commune": city}})


In [110]:
librariesInParis = table.aggregate([
    { "$match": {
        "fields.commune": "paris"
    }},
    { "$group": {
        "_id": "Compteur", "Nombre de librairies à Paris": { "$sum": 1}
    }}
])

list(librariesInParis)

[{'_id': 'Compteur', 'Nombre de librairies à Paris': 615}]

### afficher tout les noms d'établissements situé 95

In [124]:
etablissementIn95 = table.aggregate([
    { "$match": {
        "fields.codepostal": { "$regex":"^95[0-9]{3}$"}
    }},
    { "$group": {
        "_id": "$fields.nometablissement"
    }}
])

list(etablissementIn95)

[{'_id': 'Médiathèque municipale François Mauriac'},
 {'_id': 'Air France. Direction de la communication.  Centre de Documentation'},
 {'_id': "Musée d'Art et d'Histoire Louis Senlecq. Bibliothèque et archives"},
 {'_id': 'Bibliothèque intercommunale Georges-Brassens'},
 {'_id': 'Médiathèque de Jouy le Moutier'},
 {'_id': "Médiathèque de l'Horloge"},
 {'_id': 'Médiathèque municipale Les Temps Modernes'},
 {'_id': 'Archives municipales'},
 {'_id': 'Bibliothèque intercommunale du Val-de-France'},
 {'_id': 'Médiathèque Maurice Genevoix'},
 {'_id': "Musée de la Renaissance - château d'Ecouen"},
 {'_id': "Bibliothèque départementale du Val d'Oise"},
 {'_id': 'Institut universitaire de technologie. Bibliothèque'},
 {'_id': "Ecole nationale supérieure de l'électronique et de ses applications. Centre de documentation"},
 {'_id': 'Médiathèque municipale Boris Vian'},
 {'_id': 'Médiathèque  Elsa-Triolet et Louis-Aragon'},
 {'_id': "Médiathèque intercommunale L'Éclipse"},
 {'_id': "Val-d'Oise"},


### requeter la bdd pour creer un dictionnaire comportant le nombre de bibliotheque ouvert par jour de la semaine

In [242]:
day_dict = dict({"lundi": 0 , "mardi": 1 , "mercredi": 2, "jeudi": 3, "vendredi": 4, "samedi": 5, "dimanche": 6})

def increment_interval_days(dictionary: dict, input: str):
    if re.search(" (au)+ ", input, re.IGNORECASE):
        split_string = input.split()
        au_index = split_string.index("au")
        
        start_day = split_string[au_index - 1].lower()
        end_day = split_string[au_index + 1].lower()
        
        isValidDay = False
        
        for i in day_dict:
            if start_day == i:
                isValidDay = True
                
        if isValidDay == False:
            return dictionary
        
        start_day_day_dict_index = day_dict[start_day]
        
        for i in range(start_day_day_dict_index + 1, 6):
            day = list(day_dict.items())[i][0]
    
            if end_day == day:
                break
                
            dictionary[day] +=1
    
    return dictionary

In [243]:
libraryOpenedOnDay = dict({"lundi": 0, "mardi": 0, "mercredi": 0, "jeudi": 0, "vendredi": 0, "samedi": 0, "dimanche": 0})

for doc in table.find(no_cursor_timeout=True):
    fields = doc.get('fields')
    openedHours = str(fields.get('heuresouverture'))
    
    libraryOpenedOnDay = increment_interval_days(libraryOpenedOnDay, openedHours)
            
    for i in libraryOpenedOnDay:
        if re.search(i, openedHours, re.IGNORECASE):
            libraryOpenedOnDay[i] +=1

print(libraryOpenedOnDay)

{'lundi': 624, 'mardi': 848, 'mercredi': 859, 'jeudi': 771, 'vendredi': 929, 'samedi': 491, 'dimanche': 70}


### certains jours sont bien plus representé que d'autres pourquoi ?

# TP

### le TP consiste a :
  - acquerir de la donnees se mettant a jour assez régulièrement via une api ou du scrapping
    ex: (meteo, bourse, crypto-monaie ect.)
  - stocker cette données dans votre base mongodb rajouter la date precise si ce n'est pas dans vos data
  - mettre en place un outils permetant de visualiser ses data par rapport a la date
    ex:(streamlit, flask ect.)